# Project 3, Part 4, Use neo4j to calculate most efficient path from AGM main location to additional pick up locations

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering



In [3]:
import neo4j

import pandas as pd

from IPython.display import display

In [5]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [6]:
session = driver.session(database="neo4j")

In [8]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.create('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

In [9]:
my_neo4j_shortest_path('depart Ashby', 'arrive Balboa Park')


--------------------------------
   Total Cost:  2100
   Minutes:  35.0
--------------------------------
depart Ashby, 0, 0
red Ashby, 0, 0
red MacArthur, 240, 240
red 19th Street, 180, 420
red 12th Street, 120, 540
red West Oakland, 300, 840
red Embarcadero, 420, 1260
red Montgomery Street, 60, 1320
red Powell Street, 120, 1440
red Civic Center, 60, 1500
red 16th Street Mission, 180, 1680
red 24th Street Mission, 120, 1800
red Glen Park, 180, 1980
red Balboa Park, 120, 2100
arrive Balboa Park, 0, 2100


In [10]:
my_neo4j_shortest_path('depart Ashby', 'arrive Powell Street')


--------------------------------
   Total Cost:  1440
   Minutes:  24.0
--------------------------------
depart Ashby, 0, 0
red Ashby, 0, 0
red MacArthur, 240, 240
red 19th Street, 180, 420
red 12th Street, 120, 540
red West Oakland, 300, 840
red Embarcadero, 420, 1260
red Montgomery Street, 60, 1320
red Powell Street, 120, 1440
arrive Powell Street, 0, 1440


In [11]:
my_neo4j_shortest_path('depart Ashby', 'arrive Pittsburg Center')


--------------------------------
   Total Cost:  3059
   Minutes:  51.0
--------------------------------
depart Ashby, 0, 0
red Ashby, 0, 0
red MacArthur, 240, 240
yellow MacArthur, 59, 299
yellow Rockridge, 240, 539
yellow Orinda, 300, 839
yellow Lafayette, 300, 1139
yellow Walnut Creek, 300, 1439
yellow Pleasant Hill, 120, 1559
yellow Concord, 360, 1919
yellow North Concord, 180, 2099
yellow Pittsburg, 360, 2459
yellow Pittsburg Center, 600, 3059
arrive Pittsburg Center, 0, 3059


In [12]:
my_neo4j_shortest_path('depart Ashby', 'arrive Bay Fair')


--------------------------------
   Total Cost:  1740
   Minutes:  29.0
--------------------------------
depart Ashby, 0, 0
orange Ashby, 0, 0
orange MacArthur, 240, 240
orange 19th Street, 180, 420
orange 12th Street, 120, 540
orange Lake Merritt, 180, 720
orange Fruitvale, 300, 1020
orange Coliseum, 240, 1260
orange San Leandro, 240, 1500
orange Bay Fair, 240, 1740
arrive Bay Fair, 0, 1740
